## Skab database

In [14]:

import sqlite3

conn = sqlite3.connect('database.db')
c = conn.cursor()


c.execute("""CREATE TABLE IF NOT EXISTS articles (
            headline text,  
            URL text UNIQUE,
            date text,
            is_nyhed integer,
            type_article text,
            text_body text,
            countries text,
            KM_index integer, 
            language text,
            media text,
            processed integer
            )""")


conn.commit()
conn.close()



## Hjælpefunktioner

In [ ]:
# GET COLUMN NAMES

conn = sqlite3.connect('database.db')
c = conn.execute('select * from articles')

# Fetch and print column names
column_names = [description[0] for description in c.description]
print("Column Names:", column_names)

conn.close()

Column Names: ['headline', 'URL', 'date', 'is_nyhed', 'type_article', 'text_body', 'countries', 'KM_index', 'language', 'media', 'processed']


In [3]:
# GET 10 ITEMS
import sqlite3

conn = sqlite3.connect('database.db')
c = conn.execute('SELECT language FROM articles')
#WHERE media = "DR"'
rows = c.fetchmany(15)
for row in rows:
    print(row)

conn.close()

('German',)
('German',)
('German',)
('English',)
('English',)
('English',)
('English',)
('English',)
('Danish',)
('Danish',)
('Danish',)
('Danish',)
('Danish',)
('Danish',)
('Danish',)


In [ ]:
# DELETE ALL DATA
import sqlite3

conn = sqlite3.connect('database.db')
c = conn.cursor()

# Delete all rows from the table
c.execute('DELETE FROM articles')

# Commit the changes
conn.commit()
conn.close()

In [2]:
#  NUMBER OF ITEMS
import sqlite3

conn = sqlite3.connect('database.db')
c = conn.cursor()

# Delete all rows from the table
c.execute('SELECT * FROM articles')
print(len(c.fetchall()))

# Commit the changes
conn.commit()
conn.close()

118


In [1]:
# set all items to not processed
import sqlite3

conn = sqlite3.connect('database.db')
c = conn.cursor()

c.execute("UPDATE articles SET processed = 0")


# Commit the changes
conn.commit()
conn.close()

## Find unikke lande i artiklen

In [ ]:
import spacy
import demonym

# --- SETUP (RUN ONCE) ---
MODEL_MAP = {
    "Danish": "da_core_news_sm",
    "English": "en_core_web_sm",
    "German": "de_core_news_sm"
}

LOOKUP_MAP = {
    "Danish": demonym.DANISH, 
    "English": demonym.ENGLISH, 
    "German": demonym.GERMAN
}


loaded_models = {}
for lang, model_name in MODEL_MAP.items():
    loaded_models[lang] = spacy.load(model_name)
        
   

Loading spaCy models...
-> 'da_core_news_sm' for Danish loaded.


/Users/johanandersen/miniconda3/envs/scrape/lib/python3.12/site-packages/spacy/util.py:922: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.7). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


-> 'en_core_web_sm' for English loaded.


/Users/johanandersen/miniconda3/envs/scrape/lib/python3.12/site-packages/spacy/util.py:922: UserWarning: [W095] Model 'de_core_news_sm' (3.7.0) was trained with spaCy v3.7.0 and may not be 100% compatible with the current version (3.8.7). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


-> 'de_core_news_sm' for German loaded.
All models loaded successfully.



In [ ]:

import sqlite3
import analyze


conn = sqlite3.connect('database.db')
# Set row_factory to access columns by name, which is easier to read
conn.row_factory = sqlite3.Row 
c = conn.cursor()


# We only need the URL (as a unique ID) and the text_body for analysis
c.execute('SELECT URL, text_body, language FROM articles WHERE processed = 0 AND media = "DR"')
articles_to_process = c.fetchmany(5) # fetchall later!!

   
for article in articles_to_process:
    url = article['URL']
    text = article['text_body']
    language = article['language']

    nlp = loaded_models.get(language)
    lookup = LOOKUP_MAP.get(language)
    doc = nlp(text.lower())
    countries_found = analyze.find_countries(doc, lookup)

    if countries_found:
        print(f"Updating {url} -> Countries: {countries_found}")
        c.execute("""
            UPDATE articles 
            SET countries = ?, processed = 1 WHERE URL = ?""", 
            (countries_found, url))
        
    else:
        c.execute("UPDATE articles SET processed = 1 WHERE URL = ?", (url,))

conn.commit()
print("Database has been updated successfully.")
conn.close()
        

Updating https://www.dr.dk/nyheder/kultur/en-selfie-r-i-p-kondolere-r-hvordan-udtrykker-man-sin-medfoelelse-naar-nogen-doer -> Countries: Denmark
Updating https://www.dr.dk/nyheder/udland/18-personer-savnet-efter-eksplosion-paa-ammunitionsfabrik -> Countries: United States of America
Updating https://www.dr.dk/nyheder/indland/unge-ukrainske-maend-boer-blive-hjemme-og-forsvare-deres-land-mener-df-profil -> Countries: Denmark, Poland, Russia, Ukraine
Updating https://www.dr.dk/nyheder/vejret/ekstreme-regnmaengder-i-spanien-og-mere-paa-vej -> Countries: Spain
Database has been updated successfully.
